In [19]:
import requests
from bs4 import BeautifulSoup
import re
#The basics
import numpy as np
import pandas as pd
import pickle

#Get them web sites
import requests

#Make sure slenium works
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

#Start the google driver
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
#For inserting articles into Mongodb
from pymongo import MongoClient

In [20]:
mj_first_articles = pickle.load(open("pickle_files/mj_first_articles.pkl", "rb" ))
mj_more_articles = pickle.load(open("pickle_files/mj_more_articles.pkl", "rb" ))

In [21]:


def get_article(url):
#Sends request for url
    html = requests.get(url).text
    return html

def parse_article(html):
#BeautifulSoups the article
    soup = BeautifulSoup(html, 'lxml')
    title = soup.find('h1', class_='entry-title').text
    body = soup.find('article', class_='entry-content').text

    article = {
        'title': title,
        'body': body,
        'source': 'Mother Jones',
        'num_source': 9
    }

    return article

def get_parsed_article_from_link(url):
#Runs the previous two functions on the url
    return parse_article(get_article(url))

In [22]:
#phase 1: uses request to try to BeautifulSoup links
mj_list_o_articles = []
mj_problem_articles = []
for text in mj_first_articles:
    #print(text)
    try:
        art = get_parsed_article_from_link(text.encode())
        mj_list_o_articles.append(art)

    except:
        print("Problem processing url " + text)
        problem = text
        mj_problem_articles.append(problem)
    time.sleep(2)

In [23]:
#phase 2: uses selenium to go through links
driver = webdriver.Chrome(chromedriver)
for x in mj_problem_articles:
    try:
        driver.get(x)
        time.sleep(3)
        soupy = BeautifulSoup(driver.page_source, 'lxml')
        title = soupy.find('h1', class_='entry-title').text
        body = soupy.find('article', class_='entry-content').text

        articley = {
        'title': title,
        'body': body,
        'source': 'Mother Jones',
        'num_source': 9
        }
        mj_list_o_articles.append(articley)
    except:
        pass

In [24]:
#Same as above but for expanded list, phase 1
mj_more_list_o_articles = []
mj_more_problem_articles = []
for text in mj_more_articles:
    #print(text)
    try:
        art = get_parsed_article_from_link(text.encode())
        mj_more_list_o_articles.append(art)

    except:
        print("Problem processing url " + text)
        problem = text
        mj_more_problem_articles.append(problem)
    time.sleep(2)


In [25]:
#same as above for expanded list, phase 2
driver = webdriver.Chrome(chromedriver)
for x in mj_more_problem_articles:
    try:
        driver.get(x)
        time.sleep(3)
        art = get_parsed_article_from_link(x)
        mj_more_list_o_articles.append(art)
    except:
        pass

In [26]:
all_articles = mj_list_o_articles + mj_more_list_o_articles

In [35]:
len(all_articles)

25

In [30]:
#starts client in Mongodb
client = MongoClient()
test = client.derp.test

In [31]:
#creates event and loads articles into Mongodb
db = client.events
test = db.test
test.insert_many(all_articles)


In [32]:
test.count()

58

[{'title': 'Trump’s Tweet About Stormy Daniels Backfired, and Now She’s Suing for Defamation',
  'body': '\n\nOlivier Doulier/ZUMA\n\n\nLooking for news you can trust?Subscribe to our free newsletters.\n\n\n\n\n\n\n\n\n\n\n\nStormy Daniels, the adult film star who claims she had a sexual relationship with President Donald Trump, filed a new lawsuit Monday accusing the president of making false statements that “denigrate and attack” her character.\xa0\nThe lawsuit specifically cites an April 18 tweet in which Trump referred to Daniels’ previous claims that a man physically threatened her in 2011 to stop her from publicly discussing her affair with Trump. In the tweet, Trump called those allegations a “total con job.” According to the lawsuit, Trump’s tweet attempted to portray Daniels as a “liar.” Daniels is already seeking a similar defamation judgment\xa0against Michael Cohen, the president’s longtime personal lawyer who paid Daniels $130,000 to keep her from publicly discussing the a